In [1]:
import numpy as np
import os
import sys
import tensorflow as tf
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
sys.path.append('C:/Users/sarva/Tensorflow1')
from object_detection.utils import ops as utils_ops
import cv2
import time

In [2]:
%matplotlib inline
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_utils

C:/Users/sarva/Tensorflow1\object_detection\utils\visualization_utils.py:25: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\sarva\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sarva\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sarva\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sarva\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\sarva\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\User

In [3]:
model='C:/Users/sarva/Tensorflow1/object_detection/ssd_mobilenet_v1_coco_2017_11_17'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
frozengraphpath=model+'/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
labelspath=os.path.join('C:/Users/sarva/Tensorflow1/object_detection/data','mscoco_label_map.pbtxt')

In [4]:
detection_graph=tf.Graph()
with detection_graph.as_default():
    od_graph_def= tf.GraphDef()
    with tf.gfile.GFile(frozengraphpath, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [5]:
category_index=label_map_util.create_category_index_from_labelmap(labelspath,use_display_name=True)

In [6]:
cap=cv2.VideoCapture(0)

In [7]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        ret=True
        while (ret):
            ret, image_np=cap.read()
            image_tensor=detection_graph.get_tensor_by_name('image_tensor:0')
            detection_boxes=detection_graph.get_tensor_by_name('detection_boxes:0')
            detection_scores=detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes=detection_graph.get_tensor_by_name('detection_classes:0')
            
            image_np_expanded = np.expand_dims(image_np, axis=0)
            (boxes,scores,classes)=sess.run([detection_boxes,detection_scores,detection_classes],feed_dict={image_tensor:image_np_expanded})
            vis_utils.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)
            cv2.imshow('live_detection',cv2.resize(image_np,(1280,720)))
            if cv2.waitKey(25) & 0xFF==ord('q'):
                cv2.destroyAllWindows()
                cap.release()
                break
            
            
            